In [ ]:
# Install dependencies (only needed for Colab)
!pip install -q sentence-transformers hdbscan openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import hdbscan
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

# 1. Load data

In [ ]:
# Load the dataset
file_path = "/content/ISCO task list.xlsx"  # adjust path if needed
df = pd.read_excel(file_path)

# First column = isco_08_code, second column = task
df = df.dropna().drop_duplicates()
codes = df.iloc[:,0].astype(str).tolist()
tasks = df.iloc[:,1].astype(str).tolist()

# 2. Generate Embeddings

In [ ]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(tasks, normalize_embeddings=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 3. First-pass clustering (strict HDBSCAN)

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='euclidean', min_cluster_size=3, min_samples=2)
cluster_labels = clusterer.fit_predict(embeddings)

output_df = pd.DataFrame({
    "isco_08_code": codes,
    "task": tasks,
    "cluster_id": cluster_labels
})

# Separate clusters and noise
clustered_df = output_df[output_df.cluster_id != -1]
noise_df = output_df[output_df.cluster_id == -1]

noise_embeddings = [embeddings[i] for i, l in enumerate(cluster_labels) if l == -1]
noise_codes = [codes[i] for i, l in enumerate(cluster_labels) if l == -1]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


# 4. ISCO-weighted Similarity Function

In [9]:
def compute_weighted_similarity(embeddings_subset, isco_codes, alpha=0.2, digits=2):
    """
    alpha: how much extra weight to add when ISCO code (first N digits) matches
    digits: number of leading digits of ISCO code to consider
    """
    sim = cosine_similarity(embeddings_subset)
    n = len(isco_codes)
    for i in range(n):
        for j in range(i+1, n):
            if isco_codes[i][:digits] == isco_codes[j][:digits]:
                sim[i,j] *= (1 + alpha)
                sim[j,i] *= (1 + alpha)
    return sim

weighted_sim = compute_weighted_similarity(noise_embeddings, noise_codes, alpha=0.2, digits=2)
distance_matrix = 1 - weighted_sim

# 5. Second-pass HDBSCAN (looser, with ISCO weighting)

In [12]:
second_hdbscan = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=2, min_samples=1)
# Convert distance_matrix to float64
second_labels_hdb = second_hdbscan.fit_predict(distance_matrix.astype(np.float64))

second_pass_hdb_df = pd.DataFrame({
    "isco_08_code": noise_codes,
    "task": noise_df.task.values,
    "cluster_id": second_labels_hdb
})

# 6. Final Agglomerative Clustering (coverage, with ISCO weighting)

In [15]:
agglo = AgglomerativeClustering(
    metric='precomputed',
    linkage='average',
    distance_threshold=0.6,  # adjust threshold if clusters too small/large
    n_clusters=None
)
agglo_labels = agglo.fit_predict(distance_matrix)

second_pass_agglo_df = pd.DataFrame({
    "isco_08_code": noise_codes,
    "task": noise_df.task.values,
    "cluster_id": agglo_labels
})

# 7. Save results

In [16]:
output_file = "/content/task_clusters_final_pipeline.xlsx"
with pd.ExcelWriter(output_file) as writer:
    clustered_df.to_excel(writer, sheet_name="FirstPass_StrongClusters", index=False)
    second_pass_hdb_df.to_excel(writer, sheet_name="SecondPass_HDBSCAN_ISCO", index=False)
    second_pass_agglo_df.to_excel(writer, sheet_name="Final_Agglomerative_ISCO", index=False)

print(f"Results saved to {output_file}")

Results saved to /content/task_clusters_final_pipeline.xlsx


#8. Evaluate coherence

In [17]:
def cluster_coherence(embeddings_subset, labels):
    scores = {}
    for cluster_id in set(labels):
        if cluster_id == -1:  # skip noise
            continue
        idx = [i for i, l in enumerate(labels) if l == cluster_id]
        if len(idx) < 2:
            continue
        sims = cosine_similarity([embeddings_subset[i] for i in idx])
        score = np.mean(sims[np.triu_indices(len(idx), 1)])
        scores[cluster_id] = score
    return scores

# Coherence calculations
embeddings_clustered = [embeddings[i] for i,l in enumerate(cluster_labels) if l != -1]
labels_clustered = clustered_df.cluster_id.tolist()
coherence_first = cluster_coherence(embeddings_clustered, labels_clustered)
coherence_hdb = cluster_coherence(noise_embeddings, second_labels_hdb)
coherence_agglo = cluster_coherence(noise_embeddings, agglo_labels)

def avg(c): return np.mean(list(c.values())) if len(c) > 0 else 0.0

print(f"Average coherence (First-pass): {avg(coherence_first):.3f}")
print(f"Average coherence (Second-pass HDBSCAN+ISCO): {avg(coherence_hdb):.3f}")
print(f"Average coherence (Final Agglomerative+ISCO): {avg(coherence_agglo):.3f}")

Average coherence (First-pass): 0.825
Average coherence (Second-pass HDBSCAN+ISCO): 0.732
Average coherence (Final Agglomerative+ISCO): 0.509


# 9. Save and combine all results into one dataframe


In [18]:
combined = []

# 1. First-pass HDBSCAN clusters
for cid in sorted(clustered_df.cluster_id.unique()):
    cluster_df = clustered_df[clustered_df.cluster_id == cid].copy()
    new_id = f"CORE_{cid}"
    cluster_df["unified_cluster_id"] = new_id
    cluster_df["methodology"] = "FirstPass_HDBSCAN"
    combined.append(cluster_df)

# 2. Second-pass HDBSCAN clusters (skip -1)
for cid in sorted(set(second_pass_hdb_df.cluster_id)):
    if cid == -1:
        continue
    cluster_df = second_pass_hdb_df[second_pass_hdb_df.cluster_id == cid].copy()
    new_id = f"SECONDARY_HDB_{cid}"
    cluster_df["unified_cluster_id"] = new_id
    cluster_df["methodology"] = "SecondPass_HDBSCAN_ISCO"
    combined.append(cluster_df)

# 3. Final Agglomerative clusters
# Note: Agglomerative clustering has no noise, so we include all clusters
for cid in sorted(second_pass_agglo_df.cluster_id.unique()):
    cluster_df = second_pass_agglo_df[second_pass_agglo_df.cluster_id == cid].copy()
    new_id = f"SECONDARY_AGG_{cid}"
    cluster_df["unified_cluster_id"] = new_id
    cluster_df["methodology"] = "Final_Agglomerative_ISCO"
    combined.append(cluster_df)

# Concatenate everything
combined_df = pd.concat(combined, ignore_index=True)

# Save to Excel
output_file_combined = "/content/task_clusters_combined.xlsx"
combined_df.to_excel(output_file_combined, index=False)

print(f"Combined clusters saved to {output_file_combined}")
print("Total clusters:", combined_df.unified_cluster_id.nunique())

Combined clusters saved to /content/task_clusters_combined.xlsx
Total clusters: 1290
